In [ ]:
import pymysql
import time
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# MySQL数据库连接信息
HOST = '47.99.113.216'
USER = 'root'
PASSWORD = '0o0o0owjs'
DB = 'xyh'
PORT = 3306

# 监控字段变化的表格
TABLE_NAME = 'donations'

# 上次状态记录
last_state = {}

# 邮件提醒配置
MAIL_HOST = 'smtp.qq.com'  # QQ邮箱的SMTP服务器地址
MAIL_PORT = 587            # SMTP端口，587是常用端口
MAIL_USER = '2842351706@qq.com'  # 发件人QQ邮箱
MAIL_PASSWORD = 'tavcrllglnaidcij'  # 使用生成的授权码，而不是邮箱密码
TO_EMAIL = '1923557153@qq.com'  # 收件人的邮箱地址

# 邮件发送函数
def send_email(subject, body):
    msg = MIMEMultipart()
    msg['From'] = MAIL_USER
    msg['To'] = TO_EMAIL
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'plain'))

    # 连接SMTP服务器并发送邮件
    try:
        with smtplib.SMTP(MAIL_HOST, MAIL_PORT) as server:
            server.starttls()  # 启用TLS加密
            server.login(MAIL_USER, MAIL_PASSWORD)  # 使用授权码登录
            server.sendmail(MAIL_USER, TO_EMAIL, msg.as_string())  # 发送邮件
            print("邮件发送成功！")
    except smtplib.SMTPException as e:
        print(f"SMTP错误: {e}")
        return False
    except Exception as e:
        print(f"未知错误: {e}")
        return False

# 连接到MySQL数据库
def get_db_connection():
    return pymysql.connect(
        host=HOST,
        user=USER,
        password=PASSWORD,
        db=DB,
        port=PORT,
        charset='utf8mb4'
    )

# 监控并报警
def monitor_job_status():
    global last_state
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            # 状态检测：
            query = (f" SELECT t1.id,t1.actName, alu.alumniName,alu.email as email FROM {TABLE_NAME} as t1 left join alumni alu on t1.alumniId =alu.id WHERE t1.is_delete = 0 ")
            cursor.execute(query)
            results = cursor.fetchall()

            for row in results:
                job_id, job_name,is_enable, execution_state,is_push = row
                key = f'{job_id}'

                # 检查是否需要发出警报
                if key in last_state:
                    last_is_enable, last_execution_state, last_is_push = last_state[key]

                    if last_is_enable != is_enable:
                        status = '已被启用' if is_enable == 1 else '已禁用'
                        send_email(f"Job ID {job_id} ,{job_name} 状态变更", f"作业 {job_name}: {status}")
                        
                    if last_is_push != is_push:
                        # status =' ' if last_is_push == 0 else
                        if is_push == 0:
                            send_email(f"Job ID {job_id} ,{job_name} 日志状态变更", f" {job_name}: '日志记录已被禁用'")
                            
                    if last_execution_state != execution_state:
                        status = '执行中' if execution_state == 1 else ('执行结束' if execution_state == 2 else '执行异常')
                        send_email(f"Job ID {job_id}, {job_name} 执行状态变更", f"作业 {job_name}: {status}")
                # 更新当前作业状态
                last_state[key] = (is_enable, execution_state,is_push)

    finally:
        connection.close()

# 设定监控间隔
def start_monitoring(interval=5):
    while True:
        monitor_job_status()
        time.sleep(interval)  # 每60秒监控一次

if __name__ == '__main__':
    start_monitoring()  # 启动监控，每60秒检查一次状态


In [ ]:
import pymysql
import time
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# MySQL数据库连接信息
HOST = '47.99.113.216'
USER = 'root'
PASSWORD = '0o0o0owjs'
DB = 'xyh'
PORT = 3306

# 监控字段变化的表格
TABLE_NAME = 'donations'

# 上次监控的最大时间戳
last_check_time = time.time()

# 邮件提醒配置
MAIL_HOST = 'smtp.qq.com'  # QQ邮箱的SMTP服务器地址
MAIL_PORT = 587            # SMTP端口，587是常用端口
MAIL_USER = '2842351706@qq.com'  # 发件人QQ邮箱
MAIL_PASSWORD = 'tavcrllglnaidcij'  # 使用生成的授权码，而不是邮箱密码
TO_EMAIL = '1923557153@qq.com'  # 收件人的邮箱地址

# 邮件发送函数
def send_email(subject, body):
    msg = MIMEMultipart()
    msg['From'] = MAIL_USER
    msg['To'] = TO_EMAIL
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'plain'))

    # 连接SMTP服务器并发送邮件
    try:
        with smtplib.SMTP(MAIL_HOST, MAIL_PORT) as server:
            server.starttls()  # 启用TLS加密
            server.login(MAIL_USER, MAIL_PASSWORD)  # 使用授权码登录
            server.sendmail(MAIL_USER, TO_EMAIL, msg.as_string())  # 发送邮件
            print("邮件发送成功！")
    except smtplib.SMTPException as e:
        print(f"SMTP错误: {e}")
        return False
    except Exception as e:
        print(f"未知错误: {e}")
        return False

# 连接到MySQL数据库
def get_db_connection():
    return pymysql.connect(
        host=HOST,
        user=USER,
        password=PASSWORD,
        db=DB,
        port=PORT,
        charset='utf8mb4'
    )

# 监控并报警
def monitor_donations():
    global last_check_time
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            # 获取新增记录，按时间戳排序，确保获取到最新的记录
            query = f"""
            SELECT id, alumniId, actId, donationStatus, alumniName, totalAmount, createTime 
            FROM {TABLE_NAME} 
            WHERE createTime > FROM_UNIXTIME({last_check_time})
            """
            cursor.execute(query)
            results = cursor.fetchall()

            for row in results:
                donation_id, alumni_id, act_id, donation_status, alumni_name, total_amount, create_time = row

                # 如果 donationStatus 为 1，则发送邮件
                if donation_status == 1:
                    send_email(
                        f"捐款记录提醒 - {alumni_name} ({donation_id})",
                        f"捐款人 {alumni_name}（ID: {alumni_id}）对活动（ID: {act_id}）进行了捐款，金额：{total_amount}元。捐款状态已更改为：已确认。"
                    )

            # 更新上次检查时间
            last_check_time = time.time()

    finally:
        connection.close()

# 设定监控间隔
def start_monitoring(interval=60):
    while True:
        monitor_donations()
        time.sleep(interval)  # 每 interval 秒检查一次

if __name__ == '__main__':
    start_monitoring()  # 启动监控，每60秒检查一次


### 监控捐款：发送校方和捐款者

In [2]:
import pymysql
import time
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# MySQL数据库连接信息
HOST = '47.99.113.216'
USER = 'root'
PASSWORD = '0o0o0owjs'
DB = 'xyh'
PORT = 3306

# 监控字段变化的表格
TABLE_NAME = 'donations'

# 上次监控的最大时间戳
last_check_time = time.time()

# 邮件提醒配置
MAIL_HOST = 'smtp.qq.com'  # QQ邮箱的SMTP服务器地址
MAIL_PORT = 587            # SMTP端口，587是常用端口
MAIL_USER = '2842351706@qq.com'  # 发件人QQ邮箱
MAIL_PASSWORD = 'tavcrllglnaidcij'  # 使用生成的授权码，而不是邮箱密码
SCHOOL_EMAIL = '2842351706@qq.com'

# 邮件发送函数
def send_email(subject, body, recipient_email):
    msg = MIMEMultipart()
    msg['From'] = MAIL_USER
    msg['To'] = recipient_email
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'plain'))

    # 连接SMTP服务器并发送邮件
    try:
        with smtplib.SMTP(MAIL_HOST, MAIL_PORT) as server:
            server.starttls()  # 启用TLS加密
            server.login(MAIL_USER, MAIL_PASSWORD)  # 使用授权码登录
            server.sendmail(MAIL_USER, recipient_email, msg.as_string())  # 发送邮件
            print(f"邮件发送成功！发送至 {recipient_email}")
    except smtplib.SMTPException as e:
        print(f"SMTP错误: {e}")
        return False
    except Exception as e:
        print(f"未知错误: {e}")
        return False

# 连接到MySQL数据库
def get_db_connection():
    return pymysql.connect(
        host=HOST,
        user=USER,
        password=PASSWORD,
        db=DB,
        port=PORT,
        charset='utf8mb4'
    )

# 监控并报警
def monitor_donations():
    global last_check_time
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            # 获取新增记录，并关联捐款人邮箱
            query = f"""
            SELECT t1.id, t1.alumniId, t1.actId, t1.donationStatus, t1.actName, t1.totalAmount, t1.createTime, 
                   alu.email, alu.alumniName
            FROM {TABLE_NAME} as t1 
            LEFT JOIN alumni alu ON t1.alumniId = alu.id
            WHERE t1.createTime > FROM_UNIXTIME({last_check_time})
            """
            cursor.execute(query)
            results = cursor.fetchall()

            for row in results:
                donation_id, alumni_id, act_id, donation_status, act_name, total_amount, create_time, alumni_email, alumni_name = row
                
                # 清理邮箱地址，去除多余的引号或空格
                alumni_email = alumni_email.strip().strip("'").strip('"')
                
                # 如果 donationStatus 为 1，则发送邮件
                if donation_status == 1:
                    if alumni_email:  # 确保 alumni_email 不为空
                        send_email(
                            f" 河北经贸大学 感谢您的参与",
                            f" {alumni_name} ，非常感谢您参与我校“{act_name}”活动的捐款，金额：{total_amount}元。我们将合理利用本次资源，更好的建设校园。再次感谢您的付出！祝您生活愉快。",
                            alumni_email  # 使用 alumni 的邮箱作为收件人
                        )
                    else:
                        print(f"警告：找不到 {alumni_name} 的邮箱，无法发送邮件。")

                    send_email(
                        f"捐款记录提醒 - {donation_id}",
                        f"捐款人 {alumni_name}（ID: {alumni_id}）对活动 {act_name}（ID: {act_id}）进行了捐款，金额：{total_amount}元。捐款状态已更改为：已确认。",
                        SCHOOL_EMAIL  # 使用 SCHOOL_EMAIL 邮箱作为收件人
                    )

            # 更新上次检查时间
            last_check_time = time.time()

    finally:
        connection.close()

# 设定监控间隔
def start_monitoring(interval=10):
    while True:
        monitor_donations()
        time.sleep(interval)  # 每 interval 秒检查一次

if __name__ == '__main__':
    start_monitoring()  # 启动监控，每60秒检查一次


TypeError: send_email() missing 1 required positional argument: 'recipient_email'